# Introduction to Keras and TF2.0

In [1]:
import tensorflow as tf
import cProfile

/Users/hotbaby/Library/Python/3.7/lib/python/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hotbaby/Library/Python/3.7/lib/python/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hotbaby/Library/Python/3.7/lib/python/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/hotbaby/Library/Python/3.7/lib/python/site-packages

## Basic of TF2.0

In [2]:
tf.enable_eager_execution()

> 如果`enable_eager_execution`，重启jupyter notebook重试。

In [3]:
tf.executing_eagerly()

True

Run simple matrix calculations using tf

In [4]:
x = [[2.]]
m = tf.matmul(x, x)

print('x matmul x = {}'.format(m))

x matmul x = [[4.]]


Create Tensor

In [5]:
a = tf.constant([[1, 2],
                 [3, 4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [6]:
# Boradcasting
b = tf.add(a, 1)

print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [7]:
# element-wise multiplication

print(a*b)

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [8]:
a.numpy(), b.numpy()

(array([[1, 2],
        [3, 4]], dtype=int32), array([[2, 3],
        [4, 5]], dtype=int32))

In [9]:
print(tf.matmul(a,b))

tf.Tensor(
[[10 13]
 [22 29]], shape=(2, 2), dtype=int32)


Eager execution works perfectly with Numpy

In [10]:
import numpy as np

In [11]:
c = np.multiply(a, b)
print(c)

[[ 2  6]
 [12 20]]


In [12]:
# Transfer a tensor to numpy array

a.numpy()

array([[1, 2],
       [3, 4]], dtype=int32)

## Computing Gradients

Automatic differentiatin is useful for implementing machine learning algorithm such as backpropagation for traing neural networks. During eager execution, use `tf.GradientTape` to trace operations for computing  gradients later.

In [13]:
w = tf.Variable([1.])
with tf.GradientTape() as tape:
    loss = w * w

grad = tape.gradient(loss, w)
grad

<tf.Tensor: id=34, shape=(1,), dtype=float32, numpy=array([2.], dtype=float32)>

## Train a model

In [14]:
import tensorflow.keras as keras

In [15]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [16]:
x_train.shape, y_train.shape

((60000, 28, 28), (60000,))

In [17]:
N = 10000
x_train = x_train[:N, :, :]
y_train = y_train[:N]
x_test =  x_test[:N, :, :]
y_test = y_test[:N]

In [18]:
# tf.cast?

In [19]:
# x_train.shape

In [20]:
x_train = tf.cast(x_train[..., tf.newaxis]/255, tf.float32)  # TODO
x_test = tf.cast(x_test[..., tf.newaxis]/255, tf.float32)    # TODO

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [21]:
from tensorflow.keras import layers

In [22]:
# Build the model using Seqential

minist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, [3, 3], activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(64, [3, 3], activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [23]:
# layers.Dense?

In [24]:
minist_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [25]:
# Build the model using Model
inputs = tf.keras.Input(shape=(None, None, 1), name='digits')
conv_1 = tf.keras.layers.Conv2D(16, [3, 3], activation='relu')(inputs)
conv_2 = tf.keras.layers.Conv2D(16, [3, 3], activation='relu')(conv_1)
ave_pool = tf.keras.layers.GlobalAveragePooling2D()(conv_2)
outputs = tf.keras.layers.Dense(10)(ave_pool)
minist_model_2 = tf.keras.Model(inputs=inputs, outputs=outputs)

In [26]:
minist_model_2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 16)    160       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 16)    2320      
_________________________________________________________________
global_average_pooling2d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                170       
Total params: 2,650
Trainable params: 2,650
Non-trainable params: 0
_________________________________________________________________


## Two Training methods

### Use Keras fit method

In [27]:
from tensorflow.keras import optimizers
from tensorflow.keras import losses

In [28]:
minist_model.compile(optimizer=optimizers.SGD(learning_rate=1e-3),
#                     validation_split=0.1, shuffle=True,
                    loss=losses.categorical_crossentropy,
                    metrics=['accuracy'])

In [29]:
minist_model_2.compile(loss=losses.categorical_crossentropy,
                      optimizer=optimizers.Adam(),
                      metrics=['accuracy'])

In [30]:
# minist_model.fit?

In [31]:
minist_model.fit(x_train, y_train, batch_size=128, epochs=4)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/4
10000/10000 [==============================] - 16s 2ms/sample - loss: 2.3071 - acc: 0.0883
Epoch 2/4
10000/10000 [==============================] - 16s 2ms/sample - loss: 2.2993 - acc: 0.1011
Epoch 3/4
10000/10000 [==============================] - 14s 1ms/sample - loss: 2.2894 - acc: 0.1224
Epoch 4/4
10000/10000 [==============================] - 13s 1ms/sample - loss: 2.2818 - acc: 0.1395


In [32]:
pred_test = minist_model.predict(x_test)

In [33]:
pred_test.shape

(10000, 10)

## Use TF2.0

In [34]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

In [35]:
dataset = tf.data.Dataset.from_tensor_slices(
    (tf.cast(x_train[..., tf.newaxis]/255, tf.float32), tf.cast(y_train, tf.int64)))

dataset = dataset.shuffle(1000).batch(32)

In [36]:
optimizer = optimizers.SGD(learning_rate=1e-3)
loss = losses.SparseCategoricalCrossentropy(from_logits=True)

In [37]:
for epoch in range(3):
    for batch, (images, labels) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logists = minist_model(images, training=True)
            loss_value = loss(labels, logists)

        grads = tape.gradient(loss_value, minist_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, minist_model.trainable_variables))

    print('Epoch {} finished'.format(epoch))

Epoch 0 finished
Epoch 1 finished


KeyboardInterrupt: 